In [1]:
#PublicKey Generation
def key_gen(m,n,P,Q,l,e,Po,Qo,E):
    R = m*P+n*Q
    Ec = E
    Pi = Po
    Qi = Qo
   # for i in range(0,1):
    K = (l^(e-1))*R
        ### sage
        #phi = EllipticCurveIsogeny(Ec, K)
        #Ec = phi.codomain()
        #R = phi(R)
        #Pi = phi(Pi)
        #Qi = phi(Qi)
        #### kamil
    Ec, Pi, Qi = Short_Weierstrass_Velu(E, K, Pi, Qi)
    return Ec, Pi, Qi

#PK for initiator
def key_gen_init(mA,nA):
    return key_gen(mA,nA,PA,QA,lA,eA,PB,QB,E)

#PK for responder
def key_gen_resp(mB,nB):
    return key_gen(mB,nB,PB,QB,lB,eB,PA,QA,E)

########################################################################

#Key Derivation from Own Private and Other Public information
#Input: own private key, own order, other EC, other aux points
def key_der(m,n,l,e,E,P,Q):
    R = m*P+n*Q
    Ec = E
    #for i in range(0,1):
    K = (l^(e-1))*R
        #sage
        #phi = EllipticCurveIsogeny(Ec, K)
        #Ec = phi.codomain()
        #R = phi(R)
        #kamil
    Ec, _, _ = Short_Weierstrass_Velu(Ec, K, P, Q)
    key = Short_Weierstrass_j_invariant(Ec)
    return key

#Key Derivation by initiator
def key_der_init(m,n,E,P,Q):
    return key_der(m,n,lA,eA,E,P,Q)

#Key Derivation by responder
def key_der_resp(m,n,E,P,Q):
    return key_der(m,n,lB,eB,E,P,Q)

########################################################################

#Testing and test vectors
def test_KA(mA,nA,mB,nB,E):
    In = key_gen_init(mA,nA)
    Re = key_gen_resp(mB,nB)
    kA = key_der_init(mA,nA,Re[0],Re[1],Re[2])
    kB = key_der_resp(mB,nB,In[0],In[1],In[2])
    if kA == kB:
        print("Shared key:", kA)
    else:
        return "ERROR in KA"

def Elliptic_Point_Order(P):
    i = 1
    while P * i != 0:
        P = P * i
        i += 1
    return i
    
def Short_Weierstrass_j_invariant(E):
    a4 = E.a4()
    a6 = E.a6()
    b4 = 2 * a4
    b6 = 4 * a6
    b8 = a4**2 * -1
    c4 = -24 * b4
    c6 = -216 * b6
    d = -8 * b4**3 -27 * b6**2
    j = c4**3 / d
    return j
    
def Short_Weierstrass_Velu(E, gen, P, Q):
    n = Elliptic_Point_Order(gen)
    i = 1
    g2 = []
    rp = []
    rm = []
    d = E.a4()
    e = E.a6()
    while i != n: #partitioning points
        pnt = i * gen
        if pnt * 2 == 0:
            g2.append(pnt)
        else:
            if pnt not in rp and -pnt not in rp:
                rp.append(pnt)
                rm.append(-pnt)
        i += 1
    S = g2 + rp
    v = 0
    w = 0
    dPx = 0
    dQx = 0
    dPy = 0
    dQy = 0
    for point in S:
        fq = 3 * point[0]^2 + d
        gq = -2 * point[1]
        vq = 0
        if point * 2 == 0:
            vq = fq
        else:
            vq = 2 * fq
        uq = gq ^ 2
        v += vq
        w += (uq + point[0] * vq)
        ############ isogeny part
        dPx += vq / (P[0] - point[0]) + (uq / (P[0] - point[0])**2) 
        dQx += vq / (Q[0] - point[0]) + (uq / (Q[0] - point[0])**2)
        dPy += (uq * 2*P[1] / (P[0] - point[0])**3) + (vq * (P[1] - point[1]) / (P[0] - point[0])**2) - (fq * gq / (P[0] - point[0])**2)
        dQy += (uq * 2*Q[1] / (Q[0] - point[0])**3) + (vq * (Q[1] - point[1]) / (Q[0] - point[0])**2) - (fq * gq / (Q[0] - point[0])**2) 
        ############ point image part
    phi_P = (P[0] + dPx, P[1] - dPy)
    phi_Q = (Q[0] + dQx, Q[1] - dQy)
    #print(dPx)
    d_E = d - 5 * v
    e_E = e - 7 * w
    Ec = EllipticCurve([d_E, e_E])
    return Ec, Ec(phi_P), Ec(phi_Q)

def point_image(P, vq, uq, fq, gq, point):
    dPx = vq / (P[0] - point[0]) + uq / (P[0] - point[0])^2 
    dPy = (uq * 2*P[1] / (P[0] - point[0])**3) + (vq * (P[1] - point[1]) / (P[0] - point[0])**2) - (fq * gq / (P[0] - point[0])**2)
    return dPx, dPy

#Parameters

#Curve
lA = 2
lB = 3
eA = 6
eB = 1
f = 1
p = (lA^eA)*(lB^eB)*f - 1
F.<z> = GF(p^2)
E = EllipticCurve([F(1), F(0)]) #ie. y^2=x^3+x
pt = E(46, 113)

########################################################################

#Points
#PAx = F(2374093068336250774107936421407893885897*z + 2524646701852396349308425328218203569693)
#PAy = F(1944869260414574206229153243510104781725*z + 1309099413211767078055232768460483417201)
#PA = E(PAx,PAy)
#QA = E(F(-PAx),F(z*PAy))

#PBx = F(1556716033657530876728525059284431761206*z + 1747407329595165241335131647929866065215)
#PBy = F(3456956202852028835529419995475915388483*z + 1975912874247458572654720717155755005566)
#PB = E(PBx,PBy)
#QB = E(F(-PBx),F(z*PBy))

PA=E(F(189*z+177), F(174*z+27))
QA=E(F(132*z+170), F(151*z+56))

PB=E(F(31*z+80), F(64*z+179))
QB=E(F(145), F(56*z+163))

#Example, random integer coefficients, not both divisible by lA|lB respectively
mA = 27
nA = 16

mB = 17
nB = 65

test_KA(mA,nA,mB,nB,E)

SyntaxError: invalid syntax (<ipython-input-1-4e388027b4a0>, line 148)

In [386]:
def extended_euclidean_algorithm(a, b):
    
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = b, a

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    return old_r, old_s, old_t

#print(extended_euclidean_algorithm(123, 456))

def inverse_of(n, p):
    if n == 0:
        return n
    
    gcd, x, y = extended_euclidean_algorithm(n, p)
    #print(gcd, x, y)
    assert (n * x + p * y) % p == gcd

    if gcd != 1:
        raise ValueError(f"{n} has no multiplicative inverse modulo {p}")
    return x % p

#print(inverse_of(5, 19))

class EllipticCurve():
    # Tworzenie klasy krzywej eliptycznej
    # z współczynnikami A, B i liczbą pierwszą p
    def __init__(self, a, b, p):
        self.a = a % p
        self.b = b % p
        self.p = p
    
    def a4(self):
        return self.a
    
    def a6(self):
        return self.b
    
    def __str__(self):
        return f"y^2=x^3+{self.a}x+{self.b}"


class Point():

    # Poniżej budujemy punkt i utożsamamy go z daną krzywą
    # Jeśli punkt nie jest określony, to ustawiamy x i y na nieskończoność - tzw. punkt w nieskończoności
    def __init__(self, curve, x=10**12, y=10**12):
        self.curve = curve
        self.p = curve.p
        self.x = x % curve.p
        self.y = y % curve.p

    
    def __str__(self):
        return f"Point({self.x}, {self.y} with p={self.p})"

    def __add__(self, sec_point):
        # sprawdzanie czy dodajemy element neutralny
        if sec_point == Point(self.curve):
            return self
        
        elif self == Point(self.curve):
            return sec_point
        
         # sprawdzanie czy to ten sam punkt - czyli podwojenie go
        if self.x == sec_point.x and self.y == sec_point.y:
            return self.double()
        
        dx = sec_point.x - self.x
        dy = sec_point.y - self.y

        # nie chcemy dzilić przez 0 :(
        if dx == 0:
            return Point(self.curve)
        
        # obliczanie nachylenia, lecz zamiast dzielić przez dx, to mnożymy przez jego element odwrotny        
        slope = (dy * inverse_of(dx, self.p)) % self.p
        
        x = (slope ** 2) - self.x - sec_point.x
        y = slope * x + (self.y - slope * self.x)
        x %= self.p
        y %= self.p
        
        return Point(self.curve, x, -y)
    
    def __eq__(self, sec_point):
        return self.curve == sec_point.curve and self.x == sec_point.x and self.y == sec_point.y
    
    def __ne__(self, sec_point):
        return not (self == sec_point)
        
    def __mul__(self, scalar):
        # funkcja jako mnożenie punktu przez skalar - czyli dodawanie
        
        # mnożenie przez 0 zwraca punkt w nieskończoności
        if scalar == 0:
            return Point(self.curve)
        # mnożenie przez 1 zwraca ten sam punkt
        if scalar == 1:
            return self
        
         # nie dzielimy przez 0
        if self.x == 0:
            return Point(self.curve)
        
        # sytuacja przy skalarze parzystym
        if scalar % 2 == 0:
            lambda_ = ((3 * self.x * self.x + self.curve.a) * inverse_of(2 * self.y, self.p)) % self.p
            new_x = (lambda_ * lambda_ - 2 * self.x) % self.p
            new_y = (lambda_ * (self.x - new_x) - self.y) % self.p
            return Point(self.curve, new_x, new_y) * (scalar // 2)
        
        # pamiętać o nieparzystych skalarach - wtedy należy po prostu dodać
        else:
            return self + self * (scalar - 1)
    
    def __rmul__(self, scalar): ## to samo co mul - bez rmul mnozenie nie bylo by przemienne
        
        if scalar == 0:
            return Point(self.curve)
        if scalar == 1:
            return self
         # nie dzielimy przez 0
        if self.x == 0:
            return Point(self.curve)
        
        if scalar % 2 == 0:
            lambda_ = ((3 * self.x * self.x + self.curve.a) * inverse_of(2 * self.y, self.p)) % self.p
            new_x = (lambda_ * lambda_ - 2 * self.x) % self.p
            new_y = (lambda_ * (self.x - new_x) - self.y) % self.p
            return Point(self.curve, new_x, new_y) * (scalar // 2)
        
        else:
            return self + self * (scalar - 1)

    def double(self):
        # podwajanie punktu jako liczenie pochodnej - nachylenia stycznej
        
        slope = (3 * (self.x ** 2) + self.curve.a) * inverse_of(2 * self.y, self.p)
        x = (slope ** 2) - (2 * self.x)
        y = slope * x + (self.y - slope * self.x)
        
        x %= self.p
        y %= self.p

        return Point(self.curve, x, -y)
    
    def __getitem__(self, i):
        # funkcja pomocnicza do późniejszych algorytmów

        if i == 0:
            return self.x
        elif i == 1:
            return self.y

    def opposite(self):
        # funkcja negująca
        return Point(self.curve, self.x, -self.y)    
    
E0=EllipticCurve(1, 0, 191)

##### Obliczanie rzędu punktu na krzywej

In [387]:
def Elliptic_Point_Order(P):
    i = 1
    while P * i != Point(P.curve):
        i += 1
    return i


print(Elliptic_Point_Order(Point(E0, 0, 0)))

2


##### Obliczanie niezmiennika j krzywej w postaci krótkiej Weierstrassa
$y^2 = x^3 + a_{4}x + a_{6}, gdzie\;a_{4}, a_{6} \in \mathbb{F}_{p^2}$<br>
$b_{4}=2a_{4}$,    $b_{6}=4a_{6}$<br>
$c_{4}=-24b_{4}$,    $c_{6}=-216b_{6}$<br>
$d=-8b_{4}^{3}-27b_{6}^{2}$<br>
$j=\dfrac{c_{4}^{3}}{d}$

In [438]:
def Short_Weierstrass_j_invariant(E):
    a4 = E.a4()
    a6 = E.a6()
    b4 = 2 * a4
    b6 = 4 * a6
    c4 = -24 * b4
    c6 = -216 * b6
    d = -8 * b4**3 -27 * b6**2
    j = c4**3 * inverse_of(d, E.p)
    return j

##### Wzory Velu dla krzywej w postaci krótkiej Weierstrassa
Wejście: Krzywa eliptyczna $E_1: y^2 = x^3 + dx + e$ oraz generator jądra tworzonej izogenii $R$<br> 
1. Dla wszystkich punktów wygenerowanych przez $R$, czyli wszystkich punktów z jądra izogenii: jeśli jest to punkt rzędu $2$ dodajemy go do zbioru $G_{2}$, jeśli nie to do zbioru $R_{P}$ dodajemy ten punkt a do $R_{M}$ jego element przeciwny
2. Obliczamy zbiór $S=R_{P} \cup G_{2}$ 
3. Dla każdego punktu $Q = (x_Q, y_Q) \in S$:<br> 
$f_Q = 3x^2_Q + d$<br>
$g_Q = -2y_Q$<br>
Jeśli $Q$ jest rzędu $2$ to $v_q=f_q$, w przeciwnym wypadku $v_q=2f_q$<br>
$u_Q = (g_Q)^2$<br>
4. Obliczamy wartości współczynników $v$ i $w$<br>
$v = \sum_{Q \in S} v_Q, \text{    } w = \sum_{Q \in S} (u_Q + x_Qv_Q)$
<br>
5. Współczynniki wynikowej krzywej otrzymujemy w następujący sposób:<br>
Wynikowa krzywa: $E_2: y^2 = x^3 + Dx + E$<br>
$D=d-5v$<br>
$E=e-7w$<br>
6. Obraz dowolnego punktu z wejściowej krzywej $(x, y)$ jego obraz $(\alpha, \beta)$ po przejściu przez obliczoną izogenię jest dany wzorem:<br>
$\alpha = x+ \sum_{Q \in S} (\frac{v_Q}{x - x_Q} + \frac{u_Q}{(x - x_Q)^2})$<br>
$\beta = y - \sum_{Q \in S} (u_Q \frac{2y}{(x - x_Q)^3} + v_Q\frac{y - y_Q}{(x - x_Q)^2}- \frac{f_Qg_Q}{(x - x_Q)^2})$

In [439]:
def Short_Weierstrass_Velu(E, gen, P, Q):
    n = Elliptic_Point_Order(gen)
    i = 1
    g2 = []
    rp = []
    rm = []
    d = E.a4()
    e = E.a6()
    while i != n: #partitioning points
        pnt = i * gen
        if pnt * 2 == Point(pnt.curve):
            g2.append(pnt)
        else:
            if pnt not in rp and pnt.opposite() not in rp:
                rp.append(pnt)
                rm.append(pnt.opposite)
        i += 1
    S = g2 + rp
    v = 0
    w = 0
    dPx = 0
    dQx = 0
    dPy = 0
    dQy = 0
    p = E.p
    for point in S:
        fq = (3 * point[0]**2 + d) 
        gq = (-2 * point[1]) 
        vq = 0
        if point * 2 == Point(point.curve):
            vq = fq
        else:
            vq = (2 * fq) 
        uq = (gq**2) 
        v += vq 
        w += (uq + point[0] * vq) 
        ############ isogeny part
        dPx += vq * inverse_of(P[0] - point[0], p) + uq * inverse_of((P[0] - point[0])**2, p) 
        dQx += vq * inverse_of(Q[0] - point[0], p) + uq * inverse_of((Q[0] - point[0])**2, p)
        dPy += uq * 2*P[1] * inverse_of((P[0] - point[0])**3, p) + (vq * (P[1] - point[1]) * inverse_of(((P[0] - point[0])**2), p)) - (fq * gq * inverse_of(((P[0] - point[0])**2), p))
        dQy += (uq * 2*Q[1] * inverse_of(((Q[0] - point[0])**3), p)) + (vq * (Q[1] - point[1]) * inverse_of(((Q[0] - point[0])**2), p)) - (fq * gq * inverse_of(((Q[0] - point[0])**2), p)) 
        ############ point image part
    d_E = d - 5 * v 
    e_E = e - 7 * w 
    Ec = EllipticCurve(d_E, e_E, gen.p)
    phi_P = Point(Ec, P[0] + dPx, P[1] - dPy)
    phi_Q = Point(Ec, Q[0] + dQx, Q[1] - dQy)
    return Ec, phi_P, phi_Q

##### Procedura generacji klucza z parametrów startowych, opisana w schemacie powyżej
Na potrzeby tej pokazowej implementacji algorytm ma tylko jedną rundę, z tego powodu jako jądro izogenii musimy wykorzystać nie punkt $R$, a punkt $K$, jest to punkt $R$ pomnożony przez odpowiedni skalar, by zredukować jego rząd do $2$ lub $3$. Jest to konieczne, ponieważ izogenie - jeśli chcemy uzyskać jakąś o dużym stopniu - musimy obliczać jako złożenie izogenii o małych stopniach. Dla parametrów pokazowych rząd punktu $R_A$ Alicji wynosi $64$ - czyli wymagane $l_A^{e_A}$, by zredukować go do $2$ mnożymy $R_A$ razy $32$. 

In [440]:
def key_gen(m,n,P,Q,l,e,Po,Qo,E):
    R = m*P+n*Q
    Ec = E
    Pi = Po
    Qi = Qo
    K = (l**(e-1))*R
    Ec, Pi, Qi = Short_Weierstrass_Velu(E, K, Pi, Qi)
    return Ec, Pi, Qi

##### Procedura generacji klucza z parametrów publicznych, wymienionych z drugą stroną, opisana w schemacie powyżej
$K$ służy temu samemu celowi co w komórce powyżej

In [441]:
def key_derivation(m,n,l,e,E,P,Q):
    R = m*P+n*Q
    Ec = E
    K = (l**(e-1))*R
    Ec, _, _ = Short_Weierstrass_Velu(Ec, K, P, Q)
    key = Short_Weierstrass_j_invariant(Ec)
    return key

##### Funkcje pomocnicze do pierwszej części wymiany - ustalenia kluczy publicznych, inicjujący wymianę korzysta z pierwszej funkcji (Alicja), strona odpowiadająca z drugiej (Bob)

In [442]:
def key_gen_Alice(mA,nA):
    return key_gen(mA,nA,PA,QA,lA,eA,PB,QB,E)

def key_gen_Bob(mB,nB):
    return key_gen(mB,nB,PB,QB,lB,eB,PA,QA,E)

##### Funkcje pomocnicze do drugiej części wymiany - ustalenia wspólnego tajnego klucza, inicjujący wymianę korzysta z pierwszej funkcji (Alicja), strona odpowiadająca z drugiej (Bob)

In [443]:
def key_der_Alice(m,n,E,P,Q):
    return key_derivation(m,n,lA,eA,E,P,Q)

def key_der_Bob(m,n,E,P,Q):
    return key_derivation(m,n,lB,eB,E,P,Q)

##### Funkcja symulująca działanie protokołu SIDH

In [446]:
def SIDH(mA,nA,mB,nB,E):
    In = key_gen_Alice(mA,nA)
    Re = key_gen_Bob(mB,nB)
    kA = key_der_Alice(mA,nA,Re[0],Re[1],Re[2])
    kB = key_der_Bob(mB,nB,In[0],In[1],In[2])
    if kA == kB:
        print("Wspolny tajny klucz:", kA % E.p)
    else:
        return "ERROR"

##### Przykładowa instancja protkołu dla podanych parametrów


In [449]:
lA = 2
lB = 3
eA = 6
eB = 1
f = 1
p = (lA**eA)*(lB**eB)*f - 1 #liczba pierwsza p, tu p=191
E = EllipticCurve(1, 0, p) #ie. y^2=x^3+x

PA = Point(E, 119, 127)#zalezy nad jakim cialem, musi byc rzedu lA**eA
QA = Point(E, 28, 46)#zalezy nad jakim cialem, musi byc rzedu lA**eA
PB = Point(E, 46, 78)#zalezy nad jakim cialem, musi byc rzedu lB**eB
QB = Point(E, 46, 113)#zalezy nad jakim cialem, musi byc rzedu lB**eB

mA = 27
nA = 16

mB = 17
nB = 20

SIDH(mA, nA, mB, nB, E)

Wspolny tajny klucz: 9
